# Extrair as informações desejadas.

Guarda em um dicionario de dicionarios.

**Dia_da_Semana: Disciplina: [[inicio,fim],...    ]**



Dia_da_Semana: dicionario


Disciplina: valor de dia_da_semana e também um dicionario


[[inicio,fim],...    ]: Lista de listas, valor do dicionario disciplina, tendo o inicio e o fim do horario para cada linha

In [ ]:
string = '''SEGUNDA-FEIRA
08:00 - 08:50	15000441 - T1 - Redes de Computadores	Não informado
08:50 - 09:40	15000441 - T1 - Redes de Computadores	Não informado
13:30 - 14:20	15000275 - M1 - ELETROTÉCNICA INDUSTRIAL	Não informado
14:20 - 15:10	15000275 - M1 - ELETROTÉCNICA INDUSTRIAL	Não informado
15:10 - 16:00	15000365 - M1 - TEORIA ELETROMAGNÉTICA	Não informado
16:00 - 16:50	15000365 - M1 - TEORIA ELETROMAGNÉTICA	Não informado
20:40 - 21:30	15000403 - M1 - MECÂNICA GERAL	Não informado
TERÇA-FEIRA
19:00 - 19:50	15000447 - M4 - ESTATÍSTICA BÁSICA	Não informado
19:50 - 20:40	15000447 - M4 - ESTATÍSTICA BÁSICA	Não informado
QUINTA-FEIRA
15:10 - 16:00	15000291 - T1 - ELETRÔNICA DE POTÊNCIA II	Não informado'''

In [ ]:
import re 
from collections import defaultdict


def criar_dicionario(string):
  linhas = string.split(sep = '\n')

  dicionario = defaultdict(lambda : [])


  #EXTRAIR OS DIAS 

  for x in range(len(linhas) -1):
    
    extraido = re.findall(r'^[^\d]*$',linhas[x])  #CHECA SE NÃO TEM NENHUM NÚMERO, CASO NAO TENHA, É UM DIA DA SEMANA
    if len(extraido)>0:
      dia = extraido
      linha_valida = True
      x = x+1
      disciplina_dict = defaultdict(lambda : [])

      while( linha_valida  and x< len(linhas)):
        if(len(re.findall(r'^[^\d]*$',linhas[x])) ==0 ):
          
          #EXTRAIR O NOME DA DISCIPLINA
          
          aux = linhas[x].split(sep = '-')[-1]
          aux = re.findall(r'(.*)\sNão informado',aux)
          
          disciplina = aux[0].strip()
          

          #EXTRAIR AS HORAS
          
          horas = re.findall(r'\d{2}:\d{2}',linhas[x])
          
          
          x = x+1
          disciplina_dict[disciplina].append(horas)
          
        else: 
          

          linha_valida = False
      
      dicionario[dia[0]] = dict(disciplina_dict)
      disciplina_dict = defaultdict(lambda : [])
  return dict(dicionario)

dicionario = criar_dicionario(string)  
print(dicionario)

    

    




{'SEGUNDA-FEIRA': {'Redes de Computadores': [['08:00', '08:50'], ['08:50', '09:40']], 'ELETROTÉCNICA INDUSTRIAL': [['13:30', '14:20'], ['14:20', '15:10']], 'TEORIA ELETROMAGNÉTICA': [['15:10', '16:00'], ['16:00', '16:50']], 'MECÂNICA GERAL': [['20:40', '21:30']]}, 'TERÇA-FEIRA': {'ESTATÍSTICA BÁSICA': [['19:00', '19:50'], ['19:50', '20:40']]}, 'QUINTA-FEIRA': {'ELETRÔNICA DE POTÊNCIA II': [['15:10', '16:00']]}}


# Ajustar o dicionario de dicionarios:

## Converter os dias da semana em números:

In [ ]:
#Converter os dias da semana em números


def rename_keys(d, keys):
    return dict([(keys.get(k), v) for k, v in d.items()])

conversao = {'SEGUNDA-FEIRA': 0, 'TERÇA-FEIRA':1, 'QUARTA-FEIRA':2,'QUINTA-FEIRA':3,'SEXTA-FEIRA':4,'SÁBADO':5,'DOMINGO':6, 'SABADO' : 5}

#dicionario = rename_keys(dicionario,conversao)


## Simplificar os horários :transformar horarios como ['15:10', '16:00'], ['16:00', '16:50'] -> Em apenas  [15:10,16:50]

In [ ]:


# Transformar horarios como ['15:10', '16:00'], ['16:00', '16:50'] -> Em apenas  [15:10,16:50]

from dateutil.rrule import rrule, WEEKLY, MO
from datetime import date


def ajustar(lista):
  lista2 = list()
  aux = list()

  if len(lista) == 1:
    return [(lista[0][0],lista[0][1])]

  for x in range(len(lista) -1):
    passou = False
    fim = lista[x][1]
    for y in range (x+1,len(lista)):
      if fim == lista[y][0]:
        passou = True
        fim = lista[y][1]
        
    if passou and fim not in aux:
      lista2.append( (lista[x][0],fim))
      aux.append(fim)
    else:
      if fim not in aux:
        lista2.append((lista[x][0], fim))

  return lista2


ajustar([['19:50', '20:40'],['20:40','20:40']])




[('19:50', '20:40')]

In [ ]:
teste = [['19:50', '20:40']]
len(teste)

1

# Criar calendario do 0 com icalendar


In [ ]:
!pip install icalendar

In [ ]:
#CALCULAR O PRÓXIMO DIA

from dateutil.rrule import rrule, WEEKLY, MO
from datetime import date

next_monday = rrule(freq=WEEKLY, dtstart=date.today(), byweekday=6, count=1)[0]
#next_monday.hour

In [ ]:
from datetime import datetime, timedelta
from icalendar import Calendar, Event,Alarm
import pytz


def criar_evento(cal,nome,inicio, fim): #comeco:datetime
                                        # fim: datetime

  event = Event()
  event.add('summary', nome)

  tzone = pytz.timezone('America/Sao_Paulo')

  event.add('dtstart', tzone.localize(inicio, is_dst=None))
  event.add('dtend', tzone.localize(fim, is_dst=None))
  event.add('rrule', {'freq': ['WEEKLY']})

  alarm = Alarm()
  alarm.add('action', 'DISPLAY')
  alarm.add('trigger', timedelta(minutes=-10))

  event.add_component(alarm)

  cal.add_component(event)


inicio = [2021,8,5,10,5]
fim = [2021,8,5,10,35]


def criar_ical(dicionario): 
  dicionario = rename_keys(dicionario,conversao)
  cal = Calendar()

  cal.add('prodid', '-//Google Inc//Google Calendar 70.9054//EN')
  cal.add('version', '2.0')
  

  #---------------------- Passar pelo dicionario de dicionarios, ajustar as horas e criar o evento
  for k,v in dicionario.items():
    for key, value in v.items():
      next_day = rrule(freq=WEEKLY, dtstart=date.today(), byweekday=k, count=1)[0]
      time = datetime.strptime(ajustar(value)[0][0], '%H:%M')
      end_time = datetime.strptime(ajustar(value)[0][1], '%H:%M')
      
      comeco = datetime(next_day.year,next_day.month,next_day.day, time.hour, time.minute)
      fim = datetime(next_day.year,next_day.month,next_day.day, end_time.hour, end_time.minute)

      criar_evento(cal,key,comeco, fim)
  #--------------------------



  with open('calendario_2021_2.ics', 'wb') as fw:
    fw.write(cal.to_ical())

 



criar_ical(dicionario) 